# Problem 1 - The Player
 
 Smart AI
 
 

In [1]:
import random
from ps3_hangman import *

Loading word list from file...
('  ', 55909, 'words loaded.')


In [2]:
secretWord = chooseWord(loadWords())

Loading word list from file...
('  ', 55909, 'words loaded.')


In [3]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 18 23:59:07 2019

@author: Sean Won
this version is win chance of 94.33% when i played 42127 times
"""
from pprint import pprint
import random
import string 
import collections 

WORDLIST_FILENAME = "words.txt"

def loadWords():
    
    #print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # line: string
    line = inFile.readline()
    # wordlist: list of strings
    wordlist = line.split()
    #print("  ", len(wordlist), "words loaded.")
    return wordlist

def chooseWord(wordlist):
    
    return random.choice(wordlist)

def isWordGuessed(secretWord, lettersGuessed):
        
    word = ""
    for xLetters in secretWord:
        if xLetters in lettersGuessed:
            word += xLetters
            
    if word == secretWord:
        return True
    
    else:
        return False


def getGuessedWord(secretWord, lettersGuessed):
    
    result = ""   
    
    for xLetters in secretWord:
        
        if xLetters in lettersGuessed:
            result += xLetters
            
        else:
            result += "_ "          
            
    return(result)

def getAvailableLetters(lettersGuessed):
        
    alphabet = string.ascii_lowercase    
    result =""
    
    for xLetters in alphabet:
        
        if xLetters not in lettersGuessed:
            result += xLetters
            
    return(result)
    
def isLetterGuessed(secretWord, userGuess):
    
    for xLetters in secretWord:
        
        if xLetters == userGuess:
            return(True)
            
    return(False)

def hangman(stats):
    
    wordlist = loadWords()
    secretWord = chooseWord(wordlist)
    recordOfBadGuess = ""
    recordOfGoodGuess = ""
    count = 8    
    userGuess = ""
    guessSoFar = []
    prevGuess = False
    finalResult = ""
    #print("Welcome to the game Hangman!")
    #print("I am thinking of a word that is", len(secretWord), "letters long")
    
    
    while count > 0:
        #print("-----------")
        #print("You have", count, "guesses left")
        
        remainder = getAvailableLetters(guessSoFar)
        #print("Available Letters: " + remainder)
        #new code
        finalResult = getGuessedWord(secretWord, guessSoFar)
        guess = playerAI(guessSoFar, finalResult,stats)
        #
        #print("Please guess a letter:" + guess)
        userGuess = guess.lower()        
        
        prevGuess = isLetterGuessed(guessSoFar, userGuess)
        guessSoFar.append(userGuess)
        
        finalResult = getGuessedWord(secretWord, guessSoFar)
        theLetter = isLetterGuessed(secretWord, userGuess)
        
        if prevGuess == True:
            #print("Oops! You've already guessed that letter:" + finalResult)
            prevGuess = isLetterGuessed(guessSoFar, userGuess)
            guessSoFar.append(userGuess)
            
        elif theLetter == False and count > 1:
            #print("Oops! That letter is not in my word:" + finalResult)
            count -= 1
            recordOfBadGuess += userGuess
        elif theLetter == True and count > 0:
            #print("Good guess: "+ finalResult)
            recordOfGoodGuess += userGuess
            if finalResult == secretWord:
                #print("-----------")
                #print("Congratulations, you won!")
                #print("None")
                count = 0
                return(1,len(secretWord),recordOfGoodGuess,recordOfBadGuess,secretWord)
        else:
            #print("Oops! That letter is not in my word:" + finalResult)
            #print("-----------")
            #print("Sorry, you ran out of guesses. The word was "+ secretWord + ".")
            #print("None")
            count = 0
            return(0,len(secretWord),recordOfGoodGuess,recordOfBadGuess,secretWord)

def statistic():
    inFile = open(WORDLIST_FILENAME, 'r')
    line = inFile.readline()
    wordlist = line.split()
    alphabet = string.ascii_lowercase
    numDict = {}
    alphaDict ={}
    prevNum = 0
    for word in wordlist:
        number = len(word)
        if number > prevNum:
            prevNum = number
    for totalnum in range(1,prevNum+1):
        if totalnum not in numDict:
            alphaDict ={}
            for letter in alphabet:
                alphaDict[letter] = 0
            numDict[totalnum] = alphaDict
    """  
    for aword in wordlist:
        thenumber = len(aword)
        for aletter in aword:
            numDict[thenumber][aletter] += 1  
    """
    return numDict

def playerAI(guessSoFar, finalResult,stats):
    
    alpha = 0
    placeNum = 0
    checkDict = {}
    alphabet = string.ascii_lowercase

    for posletter in finalResult:
        if posletter in alphabet:
            checkDict[placeNum] = posletter
            placeNum += 1
            alpha += 1
        elif posletter == "_":
            checkDict[placeNum] = " "
            placeNum += 1
            
    
    inFile = open(WORDLIST_FILENAME, 'r')
    line = inFile.readline()
    wordBank = line.split() 
       
    possibleChoices = []
        
    for word in wordBank:
        if len(word) == len(checkDict):
            possibleChoices.append(word)
            
    #print("number of possbile choice", len(possibleChoices))
    
    tOrF = 0
    goodChoices = []
    
    for checkword in possibleChoices:
        
        for num in range(len(checkDict)):
            if checkword[num] == checkDict[num]:
                tOrF += 1
                
        if tOrF == alpha:
            goodChoices.append(checkword)
            
        tOrF = 0
    
    #print("number of good choice", len(goodChoices))
    letterList = ''.join(goodChoices)

    for guessA in guessSoFar:
        letterList = letterList.replace(guessA, "")
    #ultimateA, freq = collections.Counter(letterList).most_common(1)[0]
    #return(ultimateA)
  
    ultimateA, freq = collections.Counter(letterList).most_common(1)[0]
    percentA = freq/len(letterList)
    
    vValue = {}
    lengthOfGuess = len(checkDict)
    total = 0
    for vletter in letterList:
        vValue[vletter] = stats[lengthOfGuess][vletter]
        total += stats[lengthOfGuess][vletter]
    ultimateB, freqs = collections.Counter(vValue).most_common(1)[0]
    if total == 0:
        percentB = 0
    else:
        percentB = freqs/total
    
    if percentA > percentB:
        return ultimateA
    else:
        return ultimateB

    

stats = statistic()
result = 0
playTime = 1
rateOfWin = 0
while rateOfWin < 100:
    playTime += 1
    win, lengthOfWord,recordOfGoodGuess,recordOfBadGuess,secretWord = hangman(stats)
    result += win
    
    for uletter in secretWord:
        stats[lengthOfWord][uletter] += 1
    """ 
    for letters in recordOfGoodGuess:
        stats[lengthOfWord][letters] += 1
    
    for letterss in recordOfBadGuess:
        stats[lengthOfWord][letterss] -= 1
    """
    rateOfWin = round((result/playTime)*100,2)
    print(playTime, "and", rateOfWin)

print(stats)